In [1]:
import os
import sys
from tqdm import tqdm

# modify sys.path for it to contain the main repo path so we can import modules such as below
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from get_coordinates import get_location
from db.db_conn import engine, session_scope, ping_db
from db.models import DimensionLocation

In [2]:
ping_db(engine)

True

In [ ]:
with session_scope() as session:
    try:
        records = session.query(DimensionLocation).where(DimensionLocation.longitude == None).all()
        # This corresponds to the SQL statement: SELECT * FROM dimension_location WHERE longitude IS NULL
        for record in tqdm(records, total=len(records), desc="Enriching locations..."):
            full_location = f"{record.city}, {record.country}, {record.continent}"
            long, lat = get_location(full_location)
            if long and lat:
                record.longitude = long
                record.latitude = lat
            else:
                print(f"Could not get coordinates for {record}")
        session.commit()
    except Exception as e:
        session.rollback()
        print(e)
        print(f"Error occured while writing results to database. Rolling back...")

Enriching locations...: 100%|██████████| 341/341 [01:26<00:00,  3.95it/s]
